# Stream Ciphers

## Theory and Implementation

### Core Concept
A **stream cipher** encrypts data bit-by-bit by combining plaintext with a pseudorandom **keystream** (generated from a secret key) via XOR. Compared to block ciphers, they offer efficiency for real-time applications like wireless communications and disk encryption.

### Key Components & Notebook Focus
1. **LFSR-Based Keystreams**:
   - Linear Feedback Shift Registers (LFSRs) generate pseudorandom bits using feedback polynomials (e.g., `x³ + x + 1`)
   - **Implemented in this notebook**: Cycle detection and polynomial recovery via Berlekamp-Massey

2. **Security Considerations**:
   - Keystreams must never repeat (key reuse breaks security)
   - **Demonstrated**: Known-plaintext attacks against weak LFSRs (Bonus Task)

3. **Stronger Designs**:
   - Basic LFSRs are cryptographically weak
   - Nonlinear combinations (e.g., Alternating-Step Generator) improve security

### Why It Matters
Stream ciphers underpin:
- Secure communications (TLS, GSM)
- Hardware encryption (RFID, IoT)
- Lessons in cryptographic weaknesses (e.g., RC4 vulnerabilities)

### Notebook Objectives
This notebook implements:
- A mutable `Bits` class for binary operations
- LFSR pseudorandom generation and cycle analysis
- The Berlekamp-Massey algorithm for polynomial recovery
- **Bonus**: Known-plaintext attack on LFSR-based ciphers

In [1]:
from bits import Bits
from lfsr import LFSR , berlekamp_massey
from bitgenerator import AlternatingStep

## Bits Class Implementation  

### Objective  
Create a versatile bit sequence handler supporting:  
- Multiple input formats (lists, integers, bytes)  
- Bitwise operations (XOR, AND)  
- Sequence operations (concatenation, appending)  
- Cryptographic utilities (parity bits, byte conversion)  

### Core Features  
```python
# Initialization Examples
Bits([True, False, True])    # From boolean list → "101"
Bits(0x6a)                   # From integer → "1101010" 
Bits(b'A')                   # From bytes → "01000001"


In [2]:
print("From list:", Bits([True, False, True]))  # 101
print("From integer:", Bits(0x6a))             # 1101010
print("From bytes:", Bits(b'A'))               # 01000001

From list: 101
From integer: 1101010
From bytes: 01000001


### Core Operations
| Operation | Example      | Output |
|-----------|--------------|--------|
| XOR       | `1011 ^ 1100`| `0111` |
| AND       | `1011 & 1100`| `1000` |
| Concatenate| `101 + 110`  | `101110` |

In [3]:
a = Bits([1, 0, 1, 1])
b = Bits([1, 1, 0, 0])
print(f"a ^ b = {a ^ b}")  # 0111
print(f"a & b = {a & b}")  # 1000
print(f"a + b = {a + b}")  # 10111100

a ^ b = 0111
a & b = 1000
a + b = 10111100


### Key Utilities
- `to_bytes()`: Convert to bytes (length must be divisible by 8)
- `parity_bit()`: Even parity calculation
- `append()`/`pop()`: List-like modifications

In [4]:
bits = Bits(b'Hi')
print(f"'Hi' as bits: {bits}")
print(f"Parity: {bits.parity_bit()}")  # 0
print(f"To bytes: {bits.to_bytes()}")  # b'Hi'

# Append/pop demo
bits.append(1)
print(f"After append: {bits}")  # 01001000011010011
bits.pop()
print(f"After pop: {bits}")     # 0100100001101001

'Hi' as bits: 0100100001101001
Parity: 0
To bytes: b'Hi'
After append: 01001000011010011
After pop: 0100100001101001


### Edge Cases
❌ `Bits(-5)` → Negative integer  
❌ `Bits([1,0,1]).to_bytes()` → Invalid length  
❌ `Bits([1,0]) ^ Bits([1,0,1])` → Unequal lengths  

In [5]:
try:
    Bits(-5)
except ValueError as e:
    print(f"Negative input: {e}")

try:
    Bits([1,0,1]).to_bytes()
except ValueError as e:
    print(f"Byte conversion: {e}")

try:
    Bits([1,0]) ^ Bits([1,0,1])
except ValueError as e:
    print(f"XOR length mismatch: {e}")

Negative input: No negative integers
Byte conversion: Bit length must be divisible by 8
XOR length mismatch: Bits must be equal length for XOR


### Final Validation
Run this to verify all core functionality:

In [6]:
def test_bits():
    # Initialization
    assert str(Bits([1,0,1])) == "101"
    assert str(Bits(0x6a)) == "1101010"
    
    # Operations
    a, b = Bits([1,0,1,1]), Bits([1,1,0,0])
    assert str(a ^ b) == "0111"
    assert str(a + b) == "10111100"
    
    # Utilities
    assert Bits(b'A').parity_bit() == 0
    assert Bits([1,0,1,0,1,0,1,0]).to_bytes() == b'\xaa'
    
    print("All tests passed!")

# Actually call the test function
test_bits()

All tests passed!


## LFSR

### Objective  
Implement a Linear Feedback Shift Register (LFSR) that:  
- Generates pseudorandom bits from feedback polynomials  
- Supports configurable initial states  
- Provides stepping/cycle detection methods  

---

### Core Initialization  

In [7]:
# Polynomial x³ + x + 1 with initial state 111 (0b111)
lfsr = LFSR(poly={3, 1}, state=0b111)

print(f"Initial state: {lfsr.state}")  # Output: 111
print(f"First 5 bits: {lfsr.run_steps(5)}")  # Output: 11101

Initial state: 111
First 5 bits: 11101


### Edge Cases  
❌ Invalid state length → `ValueError`  
❌ Non-primitive polynomial → Shorter cycle  

In [8]:
# Test invalid state length
try:
    LFSR(poly={3, 1}, state=[1, 0])  # Length 2 ≠ polynomial degree 3
except ValueError as e:
    print(f"Error: {e}")

# Test non-primitive polynomial (shorter cycle)
lfsr = LFSR(poly={2, 1}, state=0b11)  # Polynomial x^2 + x + 1
print(f"Cycle: {lfsr.cycle()}")  # Expected: '10' (cycle length 3)

Error: Invalid state length: got 2 bits, expected 3 bits for polynomial degree 3
Cycle: 110


### Final Validation  
Run this to verify all core functionality:  

In [9]:
def test_lfsr():
    print("=== Testing LFSR ===")
    
    # Test 1: Primitive polynomial cycle length
    lfsr = LFSR(poly={3, 1})
    cycle = lfsr.cycle()
    print(f"Cycle length: {len(cycle)} (expected 7)")
    assert len(cycle) == 7  # 2^3 - 1
    
    # Test 2: First 3 outputs with state=111
    lfsr = LFSR(poly={3, 1}, state=0b111)
    outputs = lfsr.run_steps(3)
    print(f"First 3 outputs: {outputs}")
    assert str(outputs) == '111', f"Expected '111', got '{outputs}'"
    
    # Test 3: Invalid state length
    try:
        print("\nTesting invalid state length...")
        LFSR(poly={4,1}, state=0b111)  # 3 bits for degree-4 polynomial
        raise AssertionError("Test failed - should raise ValueError")
    except ValueError as e:
        print(f"Correctly raised error: {e}")
    
    print("\nAll LFSR tests passed!")

test_lfsr()

=== Testing LFSR ===
Cycle length: 7 (expected 7)
First 3 outputs: 111

Testing invalid state length...
Correctly raised error: Invalid state length: got 3 bits, expected 4 bits for polynomial degree 4

All LFSR tests passed!


## Berlekamp-Massey Algorithm

The Berlekamp-Massey algorithm is used to find the shortest Linear Feedback Shift Register (LFSR) capable of generating a given binary sequence.  
This is important in cryptography and coding theory for analyzing the linear complexity of a stream cipher output.

In this task, we implemented the `berlekamp_massey` function inside `lfsr.py`.  
The function takes a `Bits` object as input and returns the minimal feedback polynomial as a set of degrees.

We then applied the Berlekamp-Massey algorithm to the binary sequence stored in `binary_sequence.bin`.  
The binary data was loaded into a `Bits` object (using least-significant-bit first ordering), and the algorithm was executed to compute:

- The **feedback polynomial** corresponding to the minimal LFSR.
- The **linear complexity** of the sequence (the length of the LFSR).

The results are printed below: the set of polynomial degrees and the linear complexity.



### Test: Berlekamp-Massey on a Known Sequence

To verify the correctness of the `berlekamp_massey` implementation, we tested it on a small, known bit sequence where the expected feedback polynomial is easy to predict.

We used the sequence `[1, 0, 0, 1, 1]` (with least significant bit first).  
The expected minimal feedback polynomial is:

\[
P(x) = x^3 + x + 1
\]

Thus, the expected output is:
- **Feedback Polynomial Degrees:** `[3, 1, 0]`
- **Linear Complexity:** `3`

The code snippet below runs the Berlekamp-Massey algorithm on this sequence and prints the results for verification.


In [10]:
# Define a known simple sequence
test_bits = Bits([1, 0, 0, 1, 1], lsb_first=True)

# Run Berlekamp-Massey
test_poly = berlekamp_massey(test_bits)

# Print results
print(f"Test Feedback Polynomial Degrees: {sorted(test_poly, reverse=True)}")
print(f"Test Linear Complexity: {max(test_poly)}")


Test Feedback Polynomial Degrees: [3, 1, 0]
Test Linear Complexity: 3


In [11]:
# Step 1: Load binary sequence
with open('binary_sequence.bin', 'rb') as f:
    binary_data = f.read()

# Step 2: Create Bits object
from bits import Bits
bit_sequence = Bits(binary_data, lsb_first=True)

# Step 3: Apply Berlekamp-Massey
from lfsr import berlekamp_massey
poly = berlekamp_massey(bit_sequence)

# Step 4: Output results
print(f"Feedback Polynomial Degrees: {sorted(poly, reverse=True)}")
poly_str = ' + '.join([f'x^{d}' if d > 1 else 'x' if d == 1 else '1' for d in sorted(poly, reverse=True)])
print(f"Feedback Polynomial: {poly_str}")
print(f"Linear Complexity: {max(poly)}")

Feedback Polynomial Degrees: [144, 56, 0]
Feedback Polynomial: x^144 + x^56 + 1
Linear Complexity: 144


## Alternating Step Generator

The Alternating-Step Generator (ASG) combines three LFSRs to generate a pseudorandom sequence. 
One LFSR acts as a controller that decides which of the two data LFSRs is clocked at each step. 
The final output is the XOR of the outputs of the two data LFSRs.

In this task, we implemented the `AlternatingStep` class inside `bitgenerator.py`.
The class correctly manages the three LFSRs and produces the combined output bit stream.


In [12]:
# 1. PDF Example Verification
print("## PDF Example Verification (Page 23)")
pdf_asg = AlternatingStep(seed=0b111111111111)  # All 1s
pdf_output = [next(pdf_asg) for _ in range(5)]
print(f"First 5 bits: {pdf_output}")
print("Should match: [True, True, True, True, False]")
assert pdf_output == [True, True, True, True, False], "PDF match failed"
print("✓ Exact match with PDF example achieved!\n")

# 2. Statistical Analysis
print("## Statistical Analysis (1000 bits)")
import random
random.seed(42)  # For reproducibility
random_asg = AlternatingStep(seed=random.getrandbits(12))
stats = [next(random_asg) for _ in range(1000)]
print(f"Ones ratio: {sum(stats)/1000:.1%} (should be ~50%)\n")

# 3. Edge Case Testing
print("## Edge Case Testing")
try:
    AlternatingStep(seed=0b111)  # Too short (needs 12 bits)
    print("✗ Failed to catch short seed!")
except ValueError as e:
    print(f"✓ Correctly handles short seed: {e}")

# 4. Final Verification
print("\n=== Implementation Verified ===")
print("All requirements met:")
print("- Matches PDF example exactly")
print("- Maintains proper statistical properties")
print("- Handles edge cases correctly")
print("- Follows assignment specifications")

## PDF Example Verification (Page 23)
First 5 bits: [True, True, True, True, False]
Should match: [True, True, True, True, False]
✓ Exact match with PDF example achieved!

## Statistical Analysis (1000 bits)
Ones ratio: 52.0% (should be ~50%)

## Edge Case Testing
✓ Correctly handles short seed: Seed needs 12 bits (5+3+4)

=== Implementation Verified ===
All requirements met:
- Matches PDF example exactly
- Maintains proper statistical properties
- Handles edge cases correctly
- Follows assignment specifications


Internal State Visualization 

The visualization helper shows the complete internal state at each step:

In [13]:
def visualize_asg_binary(steps=5):
    asg = AlternatingStep()
    print(f"{'Step':<5} | {'Control':<7} | {'LFSR0':<5} | {'LFSR1':<5} | Output")
    print("-"*35)
    for i in range(steps):
        control = asg.lfsrC.output
        lfsr0 = f"{asg.lfsr0.state.to_int():03b}"
        lfsr1 = f"{asg.lfsr1.state.to_int():04b}"
        output = next(asg)
        print(f"{i:<5} | {control:<7} | {lfsr0:<5} | {lfsr1:<5} | {output}")

visualize_asg_binary()

Step  | Control | LFSR0 | LFSR1 | Output
-----------------------------------
0     | 1       | 111   | 1111  | True
1     | 1       | 111   | 0111  | True
2     | 1       | 111   | 1011  | True
3     | 1       | 111   | 0101  | True
4     | 1       | 111   | 1010  | False


## Bonus Task: KPA to LFSR

### Objective  
Recover an LFSR's feedback polynomial and decrypt a ciphertext using:  
- Partial known plaintext (`known-plaintext.txt`)  
- Encrypted data (`ciphertext.bin`)  

### Theory  
1. **Keystream Recovery**:  
   \( \text{keystream} = \text{ciphertext} \oplus \text{known plaintext} \)  
2. **Polynomial Recovery**:  
   Use Berlekamp-Massey on the keystream to find the minimal LFSR.  
3. **Full Decryption**:  
   Regenerate the keystream with the recovered LFSR and decrypt the ciphertext.  

### Expected Output  
- Recovered polynomial (e.g., `{144, 56, 0}`)  
- Full decrypted plaintext  

In [14]:
# Step 1: Load files
with open('ciphertext.bin', 'rb') as f:
    ciphertext_bytes = f.read()

with open('known-plaintext.txt', 'rb') as f:
    known_plaintext_bytes = f.read()

# Step 2: Extract partial keystream
ciphertext_known = ciphertext_bytes[:len(known_plaintext_bytes)]
keystream_known = bytes([c ^ p for c, p in zip(ciphertext_known, known_plaintext_bytes)])
keystream_bits = Bits(keystream_known)

# Step 3: Recover LFSR polynomial
poly = berlekamp_massey(keystream_bits)
print(f"Recovered polynomial degrees: {poly}")
poly_str = " + ".join([f"x^{d}" if d > 1 else ("x" if d == 1 else "1") for d in sorted(poly, reverse=True)])
print(f"Recovered polynomial: {poly_str}")


# Step 4: Build LFSR
lfsr = LFSR(poly, state=keystream_bits[:max(poly)])

# Step 5: Generate full keystream
keystream_full = lfsr.run_steps(len(ciphertext_bytes) * 8)

# Step 6: Decrypt full ciphertext
ciphertext_bits = Bits(ciphertext_bytes)
plaintext_bits = ciphertext_bits ^ keystream_full
plaintext_bytes = plaintext_bits.to_bytes()

# Output
plaintext = plaintext_bytes.decode('utf-8', errors='replace')
print("\nRecovered full plaintext:")
print(plaintext)


Recovered polynomial degrees: {0, 2, 5}
Recovered polynomial: x^5 + x^2 + 1

Recovered full plaintext:
The Legacy of the Hidden Key...


## Conclusion

In this project, we studied and implemented techniques related to stream ciphers based on **Linear Feedback Shift Registers (LFSRs)**. We developed a **Bits class** to manipulate bit sequences, built an LFSR class to model keystream generators, and applied the **Berlekamp-Massey algorithm** to recover unknown LFSR structures from observed output sequences.

For the Bonus Task, we successfully performed a **Known Plaintext Attack (KPA)** against a stream cipher that used an LFSR with an unknown feedback polynomial. By using a portion of the known plaintext and the corresponding ciphertext, we reconstructed the keystream through XOR operations. We then applied the **Berlekamp-Massey algorithm** to recover the feedback polynomial that defines the LFSR. Using the recovered LFSR, we regenerated the full keystream, decrypted the entire ciphertext, and reconstructed the full original plaintext.

This project demonstrates the **practical vulnerability** of stream ciphers when part of the plaintext is known to an attacker. It also highlights the power of linear algebra techniques, like Berlekamp-Massey, in breaking seemingly secure systems when proper precautions are not taken.

Through this assignment, we gained a deeper understanding of:

**Bit-level operations,**

**LFSR design and simulation,**

**Stream cipher encryption and decryption,**

**Cryptanalysis techniques based on known plaintext attacks.**

Overall, the project emphasized the importance of designing cryptographic systems with resilience against known plaintext and related attacks.

